In [37]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [7]:

%run ../paths.py
CONST = CONST()

In [9]:
train = pd.read_csv(CONST.TRAIN_DATA)
train.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [11]:
train_copy = train.copy()
train_copy.head()

,Unnamed: 0,tweet,class
0,12761,Mannnnnn these hoes aint loyal,1
1,25141,have a fabulous weekend beautiful souls #nofil...,0
2,53167,#fathersday #dad @user,0
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0


In [51]:
%run ../src/processing/glove_utils.py

In [14]:
glove_embed = load_embed(CONST.GLOVE_100d)

In [63]:
'<user>' in glove_embed, '<number>' in glove_embed, '<phone_number>' in glove_embed, '<elong>' in glove_embed,"<allcaps>" in glove_embed

(True, True, False, True, True)

In [141]:
vocab = build_vocab(list(train_copy['tweet'].apply(lambda x:x.split())))
vocab
oov = check_coverage(vocab,glove_embed)
#oov[:10]


100%|██████████| 42364/42364 [00:02<00:00, 19069.45it/s]

100%|██████████| 97055/97055 [00:00<00:00, 103207.90it/s]


Found embeddings for 20.63% of vocab
Found embeddings for  67.83% of all text


In [142]:
train_copy['tweet_lower'] =  train_copy['tweet'].progress_apply(lambda x:x.lower())
train_copy.head()



100%|██████████| 42364/42364 [00:00<00:00, 52942.41it/s]


,Unnamed: 0,tweet,class,tweet_lower,removed_contr
0,12761,Mannnnnn these hoes aint loyal,1,mannnnnn these hoes aint loyal,mann these hoes aint loyal
1,25141,have a fabulous weekend beautiful souls #nofil...,0,have a fabulous weekend beautiful souls #nofil...,have a fabulous weekend beautiful souls nofilt...
2,53167,#fathersday #dad @user,0,#fathersday #dad @user,fathersday dad <user>
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1,rt @sheswantsthed: these hoes ain't loyal http...,rt <user> : these hoes is not loyal <url>
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,newhairâ¨ð lapis#rollen#new#hair#brond#gra...


In [143]:
vocab = build_vocab(list(train_copy['tweet_lower'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 42364/42364 [00:02<00:00, 17764.61it/s]

100%|██████████| 91513/91513 [00:01<00:00, 88253.98it/s]


Found embeddings for 23.79% of vocab
Found embeddings for  73.79% of all text


In [144]:
add_lower(glove_embed, vocab)

Added 0 words to embedding


In [145]:
%run ../src/processing/html_tag_remover.py

In [146]:
train_copy['tweet_lower'] = train_copy['tweet_lower'].apply(lambda x : strip_tags(x))

In [147]:
%run ../src/processing/contractions.py

In [148]:
train_copy['removed_contr'] = train_copy['tweet_lower'].apply(lambda x: clean_contractions(x, contraction_mapping))
train_copy.head()


,Unnamed: 0,tweet,class,tweet_lower,removed_contr
0,12761,Mannnnnn these hoes aint loyal,1,mannnnnn these hoes aint loyal,mannnnnn these hoes aint loyal
1,25141,have a fabulous weekend beautiful souls #nofil...,0,have a fabulous weekend beautiful souls #nofil...,have a fabulous weekend beautiful souls #nofil...
2,53167,#fathersday #dad @user,0,#fathersday #dad @user,#fathersday #dad @user
3,17294,RT @SheswantstheD: these hoes ain't loyal http...,1,rt @sheswantsthed: these hoes ain't loyal http...,rt @sheswantsthed: these hoes is not loyal htt...
4,29359,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,0,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...,newhairâ¨ð #lapis#rollen#new#hair#brond#gr...


In [149]:
vocab = build_vocab(list(train_copy['removed_contr'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 42364/42364 [00:02<00:00, 15641.23it/s]

100%|██████████| 91363/91363 [00:01<00:00, 81657.62it/s]


Found embeddings for 23.85% of vocab
Found embeddings for  76.25% of all text


In [150]:
oov[:10]

[('@user', 13246),
 ('2', 505),
 ('bitch.', 420),
 ('...', 412),
 ('bihday', 383),
 ('#positive', 379),
 ("father's", 277),
 (':)', 257),
 ('4', 256),
 ('3', 255)]

In [151]:
import re
def ap(text):
    text = text.group()
    return text.replace("'","")
re.sub("\w+[']s",ap,"father's,mother's,brother's,sister's,baby's father's also uncle")

'fathers,mothers,brothers,sisters,babys fathers also uncle'

In [152]:
%run ../src/processing/regular_expressions.py

In [153]:
train_copy['removed_contr'] = train_copy['removed_contr'].apply(lambda x: tweet_preprocessing(x))


In [154]:
vocab = build_vocab(list(train_copy['removed_contr'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 42364/42364 [00:01<00:00, 23854.73it/s]

100%|██████████| 69082/69082 [00:00<00:00, 108139.94it/s]


Found embeddings for 39.13% of vocab
Found embeddings for  87.52% of all text


In [155]:
%run ../src/processing/spellings.py

In [156]:
train_copy['removed_contr'] = train_copy['removed_contr'].apply(lambda x: replace_elongated_text(x))


In [157]:
vocab = build_vocab(list(train_copy['removed_contr'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 42364/42364 [00:02<00:00, 17626.58it/s]

100%|██████████| 67898/67898 [00:01<00:00, 66749.92it/s]


Found embeddings for 39.88% of vocab
Found embeddings for  87.60% of all text


In [158]:
oov[:10]

[('“', 1224),
 ('..', 713),
 ('bihday', 584),
 ('bitch.', 420),
 ('😂😂', 374),
 ('!!', 364),
 ('😂', 223),
 ('bitch,', 199),
 ('…', 180),
 ('day!', 170)]

In [186]:
punct =  "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown



In [187]:
unknown_punct(glove_embed,punct)
#train_copy['removed_contr'] = train_copy['removed_contr'].apply(lambda x: unknown_punct(x,glove_embed))
#"rupees" in glove_embed

'“ ” ’ ∞ θ ÷ α • à − β ∅ ³ π ‘ ₹ ´ ° £ € × ™ √ ² — – '

In [200]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [206]:
%run ../src/processing/special_char.py

In [207]:
train_copy['removed_contr'] = train_copy['removed_contr'].apply(lambda x: clean_special_chars(x,punct,punct_mapping))


In [208]:
vocab = build_vocab(list(train_copy['removed_contr'].apply(lambda x:x.split())))
#vocab
oov = check_coverage(vocab,glove_embed)


100%|██████████| 42364/42364 [00:02<00:00, 18482.90it/s]

100%|██████████| 47456/47456 [00:00<00:00, 100954.95it/s]


Found embeddings for 61.57% of vocab
Found embeddings for  96.20% of all text


In [204]:
oov[:100]

[('...', 996),
 ('bihday', 636),
 ('😂😂', 446),
 ('😂', 270),
 ('â\x9d¤ï¸\x8f', 122),
 ('★', 121),
 ('ð\x9f\x98\x8a', 116),
 ('ð\x9f\x98\x8d', 103),
 ('😒', 81),
 ('ð\x9f\x98\x82', 73),
 ('prayfororlando', 72),
 ('ð\x9f\x98\x94', 70),
 ('ð\x9f\x98\x81', 65),
 ('allahsoil', 63),
 ('ð\x9f\x98¢', 61),
 ('ð\x9f\x98', 59),
 ('💯', 57),
 ('ð\x9f\x92\x95', 55),
 ('erience', 55),
 ('ð\x9f\x98\x98', 55),
 ('staing', 54),
 ('ð\x9f\x98\x80', 53),
 ('😭😭', 52),
 ('ð\x9f\x98\x84', 51),
 ('impoant', 49),
 ('httâ\x80¦', 49),
 ('ð\x9f\x98\x83', 47),
 ('orlandoshooting', 45),
 ('brexit', 45),
 ('staed', 44),
 ('ð\x9f\x98\x8e', 44),
 ('htâ\x80¦', 42),
 ('loveâ\x80¦', 42),
 ('inshot', 39),
 ('😂😭', 39),
 ('spos', 38),
 ('😩', 38),
 ('hâ\x80¦', 37),
 ('💁', 36),
 ('😴', 36),
 ('ð\x9f\x98\xad', 36),
 ('â', 35),
 ('ripchristina', 34),
 ('😭', 34),
 ('â\x98\x80ï¸\x8f', 34),
 ('heabroken', 34),
 ('😏', 34),
 ('😎', 33),
 ('ð\x9f\x92\x9c', 33),
 ('altwaystoheal', 30),
 ('ð\x9f\x92\x94', 30),
 ('udtapunjab', 30),
 ('👌', 29